# Import Libs

In [1]:
import numpy  as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import models, transforms
from torch.utils.tensorboard import SummaryWriter
import pandas as pd

# Configuration

In [2]:
classes = [
    "angry", 
    "disgust", 
    "fear", 
    "happy", 
    "sad", 
    "surprise", 
    "neutral"
]

In [3]:
class Config:
    # dataset
    TRAIN_DS_PATH = './dataset/train.csv'
    VAL_DS_PATH = './dataset/val.csv'
    TEST_DS_PATH = './dataset/test.csv'
    
    # images dir
    TRAIN_IMG_DIR = './dataset/train'
    VAL_IMG_DIR  = './dataset/val'
    TEST_IMG_DIR  = './dataset/finaltest'
    
    # training hyperparams
    EPOCHS = 50
    LR = 1e-3
    BATCH_SIZE = 16
    NUM_WORKERS = 0
    SHUFFLE = True
    
    # saved model path
    MODEL_DIR = './model/'

In [4]:
cfg = Config()

In [19]:
# tensorboard writer
writer = SummaryWriter()

# Dataset

In [9]:
# reference: https://github.com/omarsayed7/Deep-Emotion/blob/master/data_loaders.py
class EmotionDataset(Dataset):
    def __init__(self, csv_file, img_dir, datatype, transform):
        '''
        Pytorch Dataset class
        params:-
                 csv_file : the path of the csv file    (train, validation, test)
                 img_dir  : the directory of the images (train, validation, test)
                 datatype : string for searching along the image_dir (train, val, test)
                 transform: pytorch transformation over the data
        return :-
                 image, labels
        '''
        self.csv_file = pd.read_csv(csv_file)
        self.labels = self.csv_file['emotion']
        self.img_dir = img_dir
        self.transform = transform
        self.datatype = datatype

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img = Image.open(self.img_dir + self.datatype + str(idx) + '.jpg')
        labels = np.array(self.labels[idx])
        labels = torch.from_numpy(labels).long()

        if self.transform :
            img = self.transform(img)
        return img, labels

In [10]:
# tranformations
transformation= transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

In [11]:
# create datasets
train_dataset = EmotionDataset(
    csv_file=cfg.TRAIN_DS_PATH, 
    img_dir=cfg.TRAIN_IMG_DIR, 
    datatype='train', 
    transform=transformation
)

validation_dataset = EmotionDataset(
    csv_file=cfg.VAL_DS_PATH, 
    img_dir=cfg.VAL_IMG_DIR, 
    datatype='val', 
    transform = transformation
)

In [12]:
# create data loaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=cfg.BATCH_SIZE, 
    shuffle =cfg.SHUFFLE, 
    num_workers=cfg.NUM_WORKERS
)

val_loader = DataLoader(
    validation_dataset, 
    batch_size=cfg.BATCH_SIZE, 
    shuffle =cfg.SHUFFLE, 
    num_workers=cfg.NUM_WORKERS
)

# Helpers

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    """Train the model for 1 epoch
    Args:
        model: nn.Module
        train_loader: train DataLoader
        criterion: callable loss function
        optimizer: pytorch optimizer
        device: torch.device
    Returns
    -------
    Tuple[Float, Float]
        average train loss and average train accuracy for current epoch
    """

    train_losses = []
    train_corrects = []
    model.train()

    # Iterate over data.
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # prediction
        outputs = model(inputs)

        # calculate loss
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # statistics
        train_losses.append(loss.item())
        train_corrects.append(torch.sum(preds == labels.data).item())
        
    train_loss = sum(train_losses)/len(train_losses)
    train_accuracy = sum(train_corrects)/len(train_loader.dataset)      

    return train_loss, train_accuracy


def val_epoch(model, val_loader, criterion, device):
    """Validate the model for 1 epoch
    Args:
        model: nn.Module
        val_loader: val DataLoader
        criterion: callable loss function
        device: torch.device
    Returns
    -------
    Tuple[Float, Float]
        average val loss and average val accuracy for current epoch
    """

    val_losses = []
    val_corrects = []
    model.eval()

    # Iterate over data
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # prediction
            outputs = model(inputs)

            # calculate loss
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # statistics
            val_losses.append(loss.item())
            val_corrects.append(torch.sum(preds == labels.data).item())
            
        val_loss = sum(val_losses)/len(val_losses)
        val_accuracy = sum(val_corrects)/len(val_loader.dataset)

    return val_loss, val_accuracy

# Train Model

In [2]:
# load pretrained model
model = models.mobilenet.mobilenet_v2(pretrained=True)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\ivand/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [ ]:
# freeze all layers
for param in model.parameters():
    param.requires_grad = False

num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, len(classes))

In [ ]:
# transfer to cuda device if any
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)